## Intento de obtener la tabla
Genera un panda con un diccionario de palabras con la lista de archivos donde aparecen

In [2]:
import os
import re
import pandas as pd
from collections import defaultdict

# Directorio donde están los archivos txt
directorio = r"C:\Users\erick\OneDrive\Escritorio\RI\24a\data"

# Diccionario para almacenar el mapeo de palabras y los archivos donde aparecen
mapeo_palabras_archivos = defaultdict(list)

# Recorre cada archivo en el directorio
for archivo in os.listdir(directorio):
    if archivo.endswith('.txt'):
        ruta_archivo = os.path.join(directorio, archivo)
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            contenido = f.read()
            # Utiliza expresiones regulares para dividir el contenido en palabras
            palabras = re.findall(r'\b\w+\b', contenido.lower())
            # Actualiza el mapeo de palabras y archivos
            for palabra in palabras:
                mapeo_palabras_archivos[palabra].append(archivo)

# Imprime el mapeo de palabras y archivos
palabras = pd.DataFrame(mapeo_palabras_archivos.items())
palabras

,0,1
0,the,"[pg100.txt, pg100.txt, pg100.txt, pg100.txt, p..."
1,project,"[pg100.txt, pg100.txt, pg100.txt, pg100.txt, p..."
2,gutenberg,"[pg100.txt, pg100.txt, pg100.txt, pg100.txt, p..."
3,ebook,"[pg100.txt, pg100.txt, pg100.txt, pg100.txt, p..."
4,of,"[pg100.txt, pg100.txt, pg100.txt, pg100.txt, p..."
...,...,...
199516,debits,[pg996.txt]
199517,quixana,"[pg996.txt, pg996.txt]"
199518,p74b,[pg996.txt]
199519,tordesillesque,[pg996.txt]


## Intento N2
Generar un diccionario booleano por cada archivo txt

In [7]:
import os
import re
import json

# Función para procesar un archivo de texto y generar el mapeo de palabras
def generar_mapeo(archivo):
    mapeo = {}
    with open(archivo, 'r', encoding='utf-8') as file:
        texto = file.read()
        # Utilizar expresiones regulares para obtener palabras
        palabras = re.findall(r'\b\w+\b', texto.lower())
        # Actualizar el mapeo con las palabras del archivo
        for palabra in palabras:
            mapeo[palabra] = True
    return mapeo

# Obtener la lista de archivos de texto en un directorio
directorio_entrada = r'C:\Users\erick\OneDrive\Escritorio\RI\24a\data'
directorio_salida = r'C:\Users\erick\OneDrive\Escritorio\RI\24a\json'

# Crear el directorio de salida si no existe
if not os.path.exists(directorio_salida):
    os.makedirs(directorio_salida)

# Procesar cada archivo de texto y generar el archivo JSON correspondiente
archivos_txt = [f for f in os.listdir(directorio_entrada) if f.endswith('.txt')]
for archivo_txt in archivos_txt:
    mapeo_palabras = generar_mapeo(os.path.join(directorio_entrada, archivo_txt))
    nombre_json = os.path.splitext(archivo_txt)[0] + '_mapeo.json'
    ruta_json = os.path.join(directorio_salida, nombre_json)
    with open(ruta_json, 'w', encoding='utf-8') as json_file:
        json.dump(mapeo_palabras, json_file, ensure_ascii=False, indent=4)